In [1]:
%display latex

In [2]:
import algorithm_states, algorithms, common

We start with a $ n \times n $ diagonal matrix $ m $.

In [3]:
def get_matrix_element(p):
    result = ZZ.random_element()
    if result % p == 0:
        result = get_matrix_element(p)
    return result

def get_nonsquare(F):
    result = F.random_element()
    while result.is_square():
        result = F.random_element()
    return ZZ(result)

p = 5
dim = 6
k = 6
m0 = diagonal_matrix([get_matrix_element(p) * (p if i < k else 1) for i in range(dim)])
F = GF(p)
if not F((-1)^(k/2) * prod(m0[i, i] / p for i in range(k))).is_square():
    m0[k - 1, k - 1] *= get_nonsquare(F)
    
pretty_print((
    m0,
    F((-1)^(k/2) * prod(m0[i, i] / p for i in range(k))),
    F((-1)^(k/2) * prod(m0[i, i] / p for i in range(k))).is_square(),
))

(
[ -5   0   0   0   0   0]         
[  0   5   0   0   0   0]         
[  0   0   5   0   0   0]         
[  0   0   0   5   0   0]         
[  0   0   0   0 -10   0]         
[  0   0   0   0   0  15], 4, True
)

Initialization of variables

In [4]:
state = algorithm_states.ModelState(m0)
k = common.ord(p, m0.determinant())

Suppose that $ k \geq 4 $. By the Chevalley-Warning theorem, we have a solution to the equation $ \frac{m_{11}}{p} x_1^2 + \frac{m_{22}}{p} x_2^2 + \frac{m_{33}}{p} x_3^2 \equiv 0 \mod p $. Then, by Lemma 4.6, we have (if we order the diagonal entries of $ m $ in a suitable way) a matrix

$ r = \begin{pmatrix}
    1 & 0 & 0 & 0 & \dots & 0\\
    \lambda_2 & 1 & 0 & 0 & \dots & 0\\
    \lambda_3 & 0 & 1 & 0 & \dots & 0\\
    0 & 0 & 0 & 1 & \dots & 0\\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & 0 & 0 & \dots & 1\\
\end{pmatrix} \quad \text{and} \quad s = \mathrm{diag} \left(\frac{1}{p}, 1, \dots, 1 \right) $

such that $ r s $ is a partial model improvement.

Now, take $ m^\prime = (r s)^T m r s $. If $ p \mid m^\prime_{11} $, we have $ p \nmid m^\prime_{1j} $ for some $ j \in \{ 2, 3 \} $. We can then add once or twice the $ i $th row and column to the first one of $ m^\prime $ to make sure that $ p \nmid m^\prime_{11} $. The result is the same as when we would have added $ p $ or $ 2 p $ to $ \lambda_i $ in $ r $. Therefore, we assume that $ p \nmid m^\prime_{11} $.

Now, since $ p^2 \nmid m_{11} $, we must have either $ p \nmid \lambda_2 $ or $ p \nmid \lambda_3 $. By, if necessary, exchanging $ m_{22} $ and $ m_{33} $, we can assume that $ p \nmid \lambda_2 $.

The following will only change the upper left 3x3 submatrix of our matrix, so we will only show what happens there. We have 

$ (r s)^T m r s = \begin{pmatrix}
    \mu_1 & \lambda_2 \frac{m_{22}}{p} & \lambda_3 \frac{m_{33}}{p} & \dots\\
    \lambda_2 \frac{m_{22}}{p} & m_{22} & 0 & \dots\\
    \lambda_3 \frac{m_{33}}{p} & 0 & m_{33} & \dots\\
    \vdots & \vdots & \vdots & \ddots
\end{pmatrix} $

with $ \mu_1 = \frac{m_{11} + \lambda_2^2 m_{22} + \lambda_3^2 m_{33}}{p^2} $. If we then diagonalize with Lemma 4.3, we obtain

$ m^\prime := \begin{pmatrix}
    \mu_1 & 0 & 0 & \dots\\
    0 & \mu_2 & 0 & \dots\\
    0 & 0 & \mu_3 & \dots\\
    \vdots & \vdots & \vdots & \ddots
\end{pmatrix} $

for $ \mu_2 = m_{22} - \frac{\lambda_2^2 m_{22}^2}{\mu_1 p^2} $, which is not divisible by $ p $, and for some value of $ \mu_3 $. Note that the determinant of the submatrix has only changed by a factor of $ \frac{1}{p^2} $, so

$ \frac{m_{11} m_{22} m_{33}}{p^2} = \mu_1 \mu_2 \mu_3 = \mu_1 \left( m_{22} - \frac{\lambda_2^2 m_{22}^2}{\mu_1 p^2} \right) \mu_3, $

and $ \mu_3 $ is divisible by $ p $. Therefore,

$ \frac{\mu_3}p = \frac{\frac{m_{11}}{p} \frac{m_{22}}p \frac{m_{33}}p}{\mu_1 \frac{m_{22}}p p - \lambda_2^2 \left(\frac{m_{22}}p\right)^2} \equiv \frac{- \frac{m_{11}}{p} \frac{m_{22}}p \frac{m_{33}}p}{\lambda_2^2 \left(\frac{m_{22}}p\right)^2} \mod p. $

Therefore,

$ (-1)^{\frac{k}{2} - 1} \prod_{i, p \mid m^\prime_{ii}} \frac{m^\prime_{ii}}{p} \equiv \frac{(-1)^{\frac{k}{2}}}{\lambda_2^2 \left(\frac{m_{22}}p\right)^2} \prod_{i, p \mid m_{ii}} \frac{m_{ii}}{p} \mod p $

Since the right hand side is a square modulo $ p $, the left hand side is a square modulo $ p $ as well. 

In [5]:
while k >= 4:
    # Find a solution
    solution = None
    if F(-state.m[2, 2] / state.m[1, 1]).is_square():
        solution = (0, 1, ZZ(F(-state.m[2, 2] / state.m[1, 1]).sqrt()))
    else:
        for x2 in range((p + 1) / 2):
            if F(-(state.m[0, 0] + x2^2 * state.m[1, 1]) / state.m[2, 2]).is_square():
                solution = (1, x2, ZZ(F(-(state.m[0, 0] + x2^2 * state.m[1, 1]) / state.m[2, 2]).sqrt()))
                break
                
    if solution[0] == 0:
        state.switch_rows(1, 0, description = "Make the first coefficient of the solution nonzero")
        solution = (solution[1], solution[0], solution[2])
        
    state.add_row(1, 0, ZZ(solution[1]), "Apply partial improvement")
    state.add_row(2, 0, ZZ(solution[2]), "Apply partial improvement")
    state.step(elementary_matrix(QQ, dim, row1 = 0, scale= 1 / p))
    
    (_, diagonal_transform, _) = algorithms.lemma_4_3(state.m, p)
    state.step(diagonal_transform, "Rediagonalize")
    
    state.step(diagonal_matrix(1 / p^((common.ord(p, state.m[i, i]) // 2)) for i in range(dim)), "Remove excess square powers of p")
    
    pretty_print("Move entries divisible by p to the front")
    while min(i for i in range(dim) if state.m[i, i] % p != 0) < max(i for i in range(dim) if state.m[i, i] % p == 0):
        i = min(i for i in range(dim) if state.m[i, i] % p != 0)
        j = max(i for i in range(dim) if state.m[i, i] % p == 0)
        state.switch_rows(j, i)
                        
    k = common.ord(p, state.m.determinant())

Make the first coefficient of the solution nonzero


[  5   0   0   0   0   0]
[  0  -5   0   0   0   0]
[  0   0   5   0   0   0]
[  0   0   0   5   0   0]
[  0   0   0   0 -10   0]
[  0   0   0   0   0  15]

Apply partial improvement


[ 25   0  10   0   0   0]
[  0  -5   0   0   0   0]
[ 10   0   5   0   0   0]
[  0   0   0   5   0   0]
[  0   0   0   0 -10   0]
[  0   0   0   0   0  15]

[  1   0   2   0   0   0]
[  0  -5   0   0   0   0]
[  2   0   5   0   0   0]
[  0   0   0   5   0   0]
[  0   0   0   0 -10   0]
[  0   0   0   0   0  15]

...

Rediagonalize


[  -14     0     0     0     0     0]
[    0   2/7     0     0     0     0]
[    0     0     1     0     0     0]
[    0     0     0     1     0     0]
[    0     0     0     0 -15/2     0]
[    0     0     0     0     0     5]

'Move entries divisible by p to the front'

[    5     0     0     0     0     0]
[    0   2/7     0     0     0     0]
[    0     0     1     0     0     0]
[    0     0     0     1     0     0]
[    0     0     0     0 -15/2     0]
[    0     0     0     0     0   -14]

[    5     0     0     0     0     0]
[    0 -15/2     0     0     0     0]
[    0     0     1     0     0     0]
[    0     0     0     1     0     0]
[    0     0     0     0   2/7     0]
[    0     0     0     0     0   -14]

On the other hand, suppose that $ k = 2 $. Then the equation tells us that $ -\frac{m_{11} m_{22}}{p^2} $ is a square modulo $ p $. Therefore, $ -\frac{m_{11}}{m_{22}} $ is also a square modulo $ p $. If we call the root $ \lambda $, we have a solution to the equation $ m_{11} 1^2 + m_{22} \lambda^2 \equiv 0 \mod p. $

Then, by Lemma 4.6 we get a partial model improvement $ t $ is, so $ \mathop{ord}_p(t^T m t) = \mathop{ord}_p(m) - 2 = 0 $, which means that $ t $ is actually a model improvement and we are done.

In [6]:
if k == 2:
    l = ZZ(F(-state.m[0, 0] / state.m[1, 1]).sqrt())
    state.add_row(1, 0, l)
    state.step(elementary_matrix(QQ, dim, row1 = 0, scale = 1 / p))
     
    (_, diagonal_transform, _) = algorithms.lemma_4_3(state.m, p)
    state.step(diagonal_transform, "Rediagonalize")
    k = common.ord(p, state.m.determinant())

[  -25   -15     0     0     0     0]
[  -15 -15/2     0     0     0     0]
[    0     0     1     0     0     0]
[    0     0     0     1     0     0]
[    0     0     0     0   2/7     0]
[    0     0     0     0     0   -14]

[   -1    -3     0     0     0     0]
[   -3 -15/2     0     0     0     0]
[    0     0     1     0     0     0]
[    0     0     0     1     0     0]
[    0     0     0     0   2/7     0]
[    0     0     0     0     0   -14]

Rediagonalize


[ -1   0   0   0   0   0]
[  0 3/2   0   0   0   0]
[  0   0   1   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0   0 2/7   0]
[  0   0   0   0   0 -14]

By induction, we obtain a model improvement with respect to $ p $, which completes the proof.

In [7]:
state.t

[-3/5 3/10    0    0 6/35 -1/5]
[   0    0  2/5 -1/5    0    0]
[   0    0 -1/5 -2/5    0    0]
[-1/5  3/5    0    0    0    0]
[   0    0    0    0 1/35 -6/5]
[-1/5 1/10    0    0 6/35 -1/5]

In [8]:
state.t.determinant().factor()

5^-3

In [9]:
state.t.transpose() * m0 * state.t

[ -1   0   0   0   0   0]
[  0 3/2   0   0   0   0]
[  0   0   1   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0   0 2/7   0]
[  0   0   0   0   0 -14]